In [175]:
import numpy as np
import pandas as pd
import gc
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import math
import lightgbm as lgb
from collections import Counter  
import time
from scipy.stats import kurtosis,iqr
import seaborn as sns
from scipy import ptp
from tqdm import tqdm
from datetime import timedelta
import datetime
from sklearn.metrics import accuracy_score, roc_auc_score,log_loss,f1_score
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
from sklearn.preprocessing import MinMaxScaler
import warnings
import os
from sklearn.utils import shuffle
from sklearn.externals import joblib
from mpl_toolkits.mplot3d import Axes3D
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import warnings
warnings.filterwarnings('ignore')

In [176]:
### 北京市重点区域信息
cols = ['id', 'area_name', 'area_type', 'Center_x', 'Center_y', 'Grid_x', 'Grid_y', 'area']
area_passenger_info = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/area_passenger_info.csv', names=cols)

In [177]:
### 读取北京市迁徙指数
migration_index = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/processed/migration_index.csv')
migration_index.iloc[0:5]

,date,migration_index
0,2020-01-17,-5
1,2020-01-18,-4
2,2020-01-19,-8
3,2020-01-20,-11
4,2020-01-21,-11


In [178]:
### 读取sub
sub = pd.read_csv('/home/poac/code/Block_predict/submit/test_submit_example.csv', names=['id', 'date/hour', 'index'])
sub = sub.merge(area_passenger_info, how='left', on=['id'])
sub['date'] = sub['date/hour'].apply(lambda x: ''.join(str(x)[0:4]+'-'+str(x)[4:6]+'-'+str(x)[6:8]))
sub['date'] = pd.to_datetime(sub['date'])
sub['hour'] = sub['date/hour'].apply(lambda x: int(str(x)[8:]))
sub = sub.drop(['date/hour'], axis=1)

In [179]:
### 读取区域人流量指数，含标签
area_passenger_index = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/processed/area_passenger_index.csv')

In [180]:
area_passenger_index.iloc[0:5]

,id,index,date,hour,area_name,area_type,Center_x,Center_y,Grid_x,Grid_y,area,label
0,1,1.8,2020-01-17,0,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,2.0
1,1,1.5,2020-01-17,1,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,1.5
2,1,1.3,2020-01-17,2,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,1.8
3,1,1.3,2020-01-17,3,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,1.3
4,1,1.7,2020-01-17,4,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,1.4


In [181]:
### 读取input_for_sub
input_for_sub = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/processed/input_for_sub.csv')

In [182]:
input_for_sub.iloc[0:5]

,id,index,date,hour,area_name,area_type,Center_x,Center_y,Grid_x,Grid_y,area,label
0,1,0.6,2020-02-07,0,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,-999.0
1,1,0.7,2020-02-07,1,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,-999.0
2,1,1.6,2020-02-07,2,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,-999.0
3,1,0.5,2020-02-07,3,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,-999.0
4,1,0.9,2020-02-07,4,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,-999.0


In [183]:
### 对训练数据集进行处理
def get_data(df):
    ### 从北京市迁徙指数中，按天数，添加当前的迁徙指数
    df = df.merge(migration_index, how='left', on=['date'])
    ### 将date变成月份和天数
    df['month'] = df['date'].apply(lambda x: int(str(x)[5:7]))
    df['day'] = df['date'].apply(lambda x: int(str(x)[8:]))
    ### 删除原来的date，id，area_name，area_type
    df  = df.drop(['date', 'id', 'area_name', 'area_type'], axis=1)
    ### 对坐标和面积做归一化
    sc = MinMaxScaler()
#     sc.fit(df[['Center_x', 'Center_y', 'Grid_x', 'Grid_y', 'area']])
#     df[['Center_x', 'Center_y', 'Grid_x', 'Grid_y', 'area']] = sc.transform(df[['Center_x', 'Center_y', 'Grid_x', 'Grid_y', 'area']])
    
    sc.fit(df[['area']])
    df[['area']] = sc.transform(df[['area']])
    ### 将label=-999的数据删除，这部分数据由于时间太靠近线上，没有标签
    df = df[df['label']!=-999].reset_index(drop=True)
    ### 将特征和label分离
    labels = np.zeros((df.shape[0], 1))
    labels = pd.DataFrame(labels)
    labels.columns = ['label']
    labels = df['label']
    ### 删除原始数据中的label
    df = df.drop(['label'], axis=1)
    ### 返回特征,标签,归一化句柄
    return df, labels, sc

In [184]:
processed, label, sc = get_data(df=area_passenger_index)

In [185]:
processed.iloc[0:5]

,index,hour,Center_x,Center_y,Grid_x,Grid_y,area,migration_index,month,day
0,1.8,0,116.256586,39.986913,116.256713,39.987525,0.017824,-5,1,17
1,1.5,1,116.256586,39.986913,116.256713,39.987525,0.017824,-5,1,17
2,1.3,2,116.256586,39.986913,116.256713,39.987525,0.017824,-5,1,17
3,1.3,3,116.256586,39.986913,116.256713,39.987525,0.017824,-5,1,17
4,1.7,4,116.256586,39.986913,116.256713,39.987525,0.017824,-5,1,17


In [186]:
### 对线上提交数据进行处理
def get_sub(df, sc):
    ### 从北京市迁徙指数中，按天数，添加当前的迁徙指数
    df = df.merge(migration_index, how='left', on=['date'])
    ### 将date变成月份和天数
    df['month'] = df['date'].apply(lambda x: int(str(x)[5:7]))
    df['day'] = df['date'].apply(lambda x: int(str(x)[8:]))
    ### 删除原来的date，id，area_name，area_type
    df  = df.drop(['date', 'id', 'area_name', 'area_type'], axis=1)
    ### 对坐标和面积做归一化
    df[['area']] = sc.transform(df[['area']])
    
#     df[['Center_x', 'Center_y', 'Grid_x', 'Grid_y', 'area']] = sc.transform(df[['Center_x', 'Center_y', 'Grid_x', 'Grid_y', 'area']])

    ### 删除原始数据中的label
    df = df.drop(['label'], axis=1)
    return df

In [187]:
input_for_sub = get_sub(df=input_for_sub, sc=sc)

In [188]:
input_for_sub.iloc[0:5]

,index,hour,Center_x,Center_y,Grid_x,Grid_y,area,migration_index,month,day
0,0.6,0,116.256586,39.986913,116.256713,39.987525,0.017824,12,2,7
1,0.7,1,116.256586,39.986913,116.256713,39.987525,0.017824,12,2,7
2,1.6,2,116.256586,39.986913,116.256713,39.987525,0.017824,12,2,7
3,0.5,3,116.256586,39.986913,116.256713,39.987525,0.017824,12,2,7
4,0.9,4,116.256586,39.986913,116.256713,39.987525,0.017824,12,2,7


In [189]:
### K折交叉验证训练模型
### 定义模型参数
params = {'num_leaves': 38,
          'min_data_in_leaf': 50,
          'objective': 'regression',
          'max_depth': -1,
          'learning_rate': 0.02,
          "min_sum_hessian_in_leaf": 6,
          "boosting": "gbdt",
          "feature_fraction": 0.9,
          "bagging_freq": 1,
          "bagging_fraction": 0.7,
          "bagging_seed": 11,
          "lambda_l1": 0.1,
          "verbosity": -1,
          "nthread": 4,
          'metric': 'mae',
          "random_state": 2019,
#           'device': 'gpu'
          }

In [190]:
def r_mean_absolute_percentage_error(y_true, y_pred):
    return np.sqrt(np.mean((y_true-y_pred)**2))

In [191]:
folds = KFold(n_splits=5, shuffle=True, random_state=2019)

In [192]:
### K 折
for fold_, (trn_idx, val_idx) in enumerate(folds.split(processed)):
    print("fold {}".format(fold_ + 1))
    trn_data = lgb.Dataset(processed.iloc[trn_idx], label=label.iloc[trn_idx])
    val_data = lgb.Dataset(processed.iloc[val_idx], label=label.iloc[val_idx])
    num_round = 1000
    clf = lgb.train(params,
                    trn_data,
                    num_round,
                    valid_sets=[trn_data, val_data],
                    verbose_eval=200,
                    early_stopping_rounds=200)
    pred = clf.predict(processed.iloc[val_idx], num_iteration=clf.best_iteration)
    print('RMSE:', r_mean_absolute_percentage_error(y_true=label.iloc[val_idx], y_pred=pred))
    print('Score:', 1/(1+r_mean_absolute_percentage_error(y_true=label.iloc[val_idx], y_pred=pred)))

fold 1
Training until validation scores don't improve for 200 rounds
[200]	training's l1: 3.99515	valid_1's l1: 3.981
[400]	training's l1: 3.40615	valid_1's l1: 3.41424
[600]	training's l1: 3.19642	valid_1's l1: 3.21919
[800]	training's l1: 3.05576	valid_1's l1: 3.09017
[1000]	training's l1: 2.92823	valid_1's l1: 2.97175
Did not meet early stopping. Best iteration is:
[1000]	training's l1: 2.92823	valid_1's l1: 2.97175
RMSE: 6.381323673766606
Score: 0.1354770559044881
fold 2
Training until validation scores don't improve for 200 rounds
[200]	training's l1: 3.97356	valid_1's l1: 4.023
[400]	training's l1: 3.38731	valid_1's l1: 3.45006
[600]	training's l1: 3.18037	valid_1's l1: 3.24855
[800]	training's l1: 3.02166	valid_1's l1: 3.09328
[1000]	training's l1: 2.90447	valid_1's l1: 2.98103
Did not meet early stopping. Best iteration is:
[1000]	training's l1: 2.90447	valid_1's l1: 2.98103
RMSE: 6.610845074763993
Score: 0.1313914539287885
fold 3
Training until validation scores don't improve 

In [193]:
### submit
submit = pd.read_csv('/home/poac/code/Block_predict/submit/test_submit_example.csv', names=['id', 'date/hour', 'index'])
submit = submit.merge(area_passenger_info, how='left', on=['id'])

In [194]:
submit.iloc[0:5]

,id,date/hour,index,area_name,area_type,Center_x,Center_y,Grid_x,Grid_y,area
0,1,2020021600,1.5,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
1,1,2020021601,1.5,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
2,1,2020021602,1.5,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
3,1,2020021603,1.5,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
4,1,2020021604,1.5,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375


In [195]:
def get_submit(df, pred):
    ### 只保留id，日期，小时
    df = df[['id', 'date/hour']]
    ### 转换日期格式
    ### 将预测结果合并
    df = pd.concat([df, pred], axis=1)
    return df

In [196]:
### 预测
pred = clf.predict(input_for_sub)
pred = pd.DataFrame(pred)
pred.columns = ['pred']

In [197]:
submit = get_submit(df=submit, pred=pred)

In [198]:
submit.iloc[0:5]

,id,date/hour,pred
0,1,2020021600,0.484860
1,1,2020021601,0.571387
2,1,2020021602,1.417291
3,1,2020021603,0.272061
4,1,2020021604,0.647198


In [199]:
### 保存
submit.to_csv('/home/poac/code/Block_predict/submit/block_predict_result.csv', index=None, header=None)